In [ ]:
# Curve dataset
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
# def train_one_epoch(model, loader, optimizer, criterion, device):
#     model.train()
#     total_loss = 0
#     correct = 0
#     for inputs, targets in loader:
#         inputs, targets = inputs.to(device), targets.to(device)

#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, targets)
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item() * inputs.size(0)
#         correct += (outputs.argmax(dim=1) == targets).sum().item()

#     avg_loss = total_loss / len(loader.dataset)
#     accuracy = correct / len(loader.dataset)
#     return avg_loss, accuracy

# def evaluate(model, loader, criterion, device):
#     model.eval()
#     total_loss = 0
#     correct = 0
#     with torch.no_grad():
#         for inputs, targets in loader:
#             inputs, targets = inputs.to(device), targets.to(device)
#             outputs = model(inputs)
#             loss = criterion(outputs, targets)
#             total_loss += loss.item() * inputs.size(0)
#             correct += (outputs.argmax(dim=1) == targets).sum().item()
#     avg_loss = total_loss / len(loader.dataset)
#     accuracy = correct / len(loader.dataset)
#     return avg_loss, accuracy

# class CuretDataset(Dataset):
#     def __init__(self, root_dir, transform=None):
#         self.root_dir = Path(root_dir)
#         self.transform = transform
#         self.image_paths = []
#         self.labels = []
#         self.classes = sorted([d.name for d in self.root_dir.iterdir() if d.is_dir()])
#         self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}

#         for cls in self.classes:
#             cls_folder = self.root_dir / cls
#             for img_file in sorted(cls_folder.glob("*.png")) + sorted(cls_folder.glob("*.jpg")) + sorted(cls_folder.glob("*.bmp")):
#                 self.image_paths.append(img_file)
#                 self.labels.append(self.class_to_idx[cls])

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         img_path = self.image_paths[idx]
#         image = Image.open(img_path).convert("L")  # grayscale
#         label = self.labels[idx]
#         if self.transform:
#             image = self.transform(image)
#         return image, label



# def get_curet_loaders(batch_size=64):
#     transform = transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize((0.1307,), (0.3081,))
#     ])
#     dataset = CuretDataset("./data/curetgrey", transform=transform)
#     train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 46, 46], generator=torch.Generator().manual_seed(42))

#     train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#     test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#     return train_loader, test_loader

def run_curet_experiment(scale_J, angle_K, kernel_size, model, lr, image_shape=(1, 200), nb_class=61):
    
    train_loader, test_loader = get_curet_loaders(batch_size=64)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(6):  # Change number of epochs as needed
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
        test_loss, test_acc = evaluate(model, test_loader, criterion, device)
        print(f"Epoch {epoch+1}: Train Acc={train_acc:.4f}, Test Acc={test_acc:.4f}")
    return model

In [10]:
import sys
import os

# Get the absolute path to ../src and append it to sys.path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
sys.path.append(src_path)
from dense import dense
import torch
from torch import nn
# Try different combinations
models = []
device = "cuda" if torch.cuda.is_available() else "cpu"
J_params = [3]
K_params = [4]
kernel_size_params = [9]
for J in J_params:
    for K in K_params:
        for ks in kernel_size_params:
            print(f"\nRunning: J={J}, K={K}, kernel={ks}")
            model = dense(J, K, (1, 200), ks, 61).to(device)
            for conv in model.sequential_conv:
                for param in conv.parameters():
                    param.requires_grad = False
            for param in model.linear.parameters():
                param.requires_grad = True
            models.append(run_curet_experiment(J, K, ks, model, 1e-3))

for model in models:
    for conv in model.sequential_conv:
        for param in conv.parameters():
            param.requires_grad = True
    for param in model.linear.parameters():
        param.requires_grad = False
    models.append(run_curet_experiment(J, K, ks, model, 1e-5))
            


Running: J=3, K=4, kernel=9
Creating filter bank with Sampling support width=8, Size=9, Angles=4 ...
Creating filter bank with Sampling support width=8, Size=17, Angles=4 ...
Creating filter bank with Sampling support width=8, Size=33, Angles=4 ...
Epoch 1: Train Acc=0.2787, Test Acc=0.3043
Epoch 2: Train Acc=0.6286, Test Acc=0.6304
Epoch 3: Train Acc=0.7693, Test Acc=0.6522
Epoch 4: Train Acc=0.8611, Test Acc=0.7826
Epoch 5: Train Acc=0.8999, Test Acc=0.8261
Epoch 6: Train Acc=0.9170, Test Acc=0.8043
Epoch 1: Train Acc=0.9319, Test Acc=0.8261


KeyboardInterrupt: 

In [ ]:
# PCA
# Needs to run 

from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
import torch

def extract_features_and_labels(model, dataloader):
    model.eval()
    features = []
    labels = []
    with torch.no_grad():
        for images, targets in dataloader:
            images = images.to(device)
            feats = model(images).view(images.size(0), -1).cpu().numpy()
            features.append(feats)
            labels.extend(targets.numpy())
    return np.vstack(features), np.array(labels)


# Try different combinations
models = []
device = "cuda" if torch.cuda.is_available() else "cpu"
J_params = [2]
K_params = [4]
kernel_size_params = [9]
for J in J_params:
for K in K_params:
    for ks in kernel_size_params:
        print(f"\nRunning: J={J}, K={K}, kernel={ks}")
        model = dense(J, K, (1, 200), ks).to(device)
        train_loader, test_loader = get_curet_loaders(batch_size=64)
        train_feats, train_labels = extract_features_and_labels(model, train_loader)
        test_feats, test_labels = extract_features_and_labels(model, test_loader)

        # Apply PCA
        pca = PCA(n_components=0.95)  # or fixed value like 100
        train_pca = pca.fit_transform(train_feats)
        test_pca = pca.transform(test_feats)

        # Train classifier (logistic regression or nearest centroid, etc.)
        clf = LogisticRegression(max_iter=1000)
        clf.fit(train_pca, train_labels)

        # Evaluate
        preds = clf.predict(test_pca)
        acc = accuracy_score(test_labels, preds)
        print(f"✅ PCA classifier test accuracy: {acc:.4f}")
